In [70]:
import os
import xml.etree.ElementTree as ET
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import warnings

In [71]:
ns = { "tei" : "http://www.tei-c.org/ns/1.0" }

In [72]:
warnings.filterwarnings("ignore")

In [67]:
#Choose name of book and category (for now: either 'graphic' or 'music')
# Other categories can be added, remember to create the corresponding folder in 'music_examples'
def extract_images(book, category):
    tree = ET.parse(f'data/{book}/{book}_tei.xml')
    root = tree.getroot()
    
    for facsimile in root.findall(".//tei:facsimile", ns):
        img_url = facsimile.find(".//tei:graphic", ns).attrib["url"]
        
        if category=='music':
            rendition = 'Music'
        elif category=='graphic':
            rendition = 'Graphic'
        else:
            print('invalid category: choose music or graphic')
            break
        for zone in facsimile.findall(f".//tei:zone[@rendition='{rendition}']", ns)[:2]:
            img_id = zone.attrib["{http://www.w3.org/XML/1998/namespace}id"]
             
            points = zone.attrib["points"]
            points = points.split()
            points = [list(map(int, p.split(','))) for p in points]        
            points = np.array(points)
            x1, y1 = points.min(axis=0)
            x2, y2 = points.max(axis=0)

            img = Image.open(f'data/{book}/{book}/' + img_url)
            
            music_region = img.crop((x1,y1,x2,y2))
            music_region.save(f'data/{book}/music_examples/{category}/{img_id}.png')


In [68]:
#Extract images for a single book
extract_images('CAP1905', category='graphic')

In [73]:
#Extract images for the whole list of books
for name in tqdm(os.listdir('data')):
    extract_images(name, category='music')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:56<00:00, 11.64s/it]
